There are 3 types of Recommendation Engines
1. Recommend the most popular items in a category: This way, there is no personalization involved. Often times, you could probably query database straight to answer this question. 

2. Classification algorithms: You can simply treat this as binary classification problem. 

3. Recommendation Algorithms: 
> 1. Content based algorithms
> 2. Collaborative Filtering algorithms
>>1. User-user collaborative filtering
>>2. Item-item collaborative filtering
>>3. Market basket analysis

This project, I am going to implement Collaborative Filtering algorithm and Content based algorithm. 



In [1]:
import pandas as pd 
